In [46]:
from bs4 import BeautifulSoup
from DB import AIModelHandler
from DB import DataFrameHandler
import requests, datetime
import pandas as pd
import json
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def getRealtimePrice(isin):
    url = f'https://polling.finance.naver.com/api/realtime?query=SERVICE_ITEM:{isin}'
    response = requests.get(url, verify=False)
    if response.status_code == 200:
        data = response.json()
        result = data['result']['areas'][0]['datas'][0]['nv']
        time = data['result']['time']
        
        return {'TIME' : datetime.datetime.fromtimestamp(time / 1000).strftime('%Y-%m-%d %H:%M:%S'),'ISIN' : isin, 'PRICE' : int(result)}

    else:
        print(f"Failed to fetch data from {url}. Status code: {response.status_code}")
        return None
    
def getIndex():
    url = f'https://ecos.bok.or.kr/api/StatisticSearch/A5P1FVP5VX625OV2NVTA/json/kr/1/100000/802Y001/D/20130101/20240101/0089000'
    response = requests.get(url, verify=False)
    if response.status_code == 200:
        data = response.json()
        result = data['StatisticSearch']['row']['DATA_VALUE']
        time = data['StatisticSearch']['row']['TIME']
        
        return {'TIME' : time,'IDX_CODE' : "KOSDAQ", 'PRICE' : result}

    else:
        print(f"Failed to fetch data from {url}. Status code: {response.status_code}")
        return None

In [47]:
getIndex()

SSLError: HTTPSConnectionPool(host='ecos.bok.or.kr', port=443): Max retries exceeded with url: /api/StatisticSearch/A5P1FVP5VX625OV2NVTA/json/kr/1/100000/802Y001/D/20130101/20240101/0089000 (Caused by SSLError(SSLError(1, '[SSL: SSLV3_ALERT_HANDSHAKE_FAILURE] sslv3 alert handshake failure (_ssl.c:992)')))

In [42]:
getRealtimePrice('005930')

{'TIME': '2023-10-01 15:07:28', 'ISIN': '005930', 'PRICE': 68400}

In [41]:
result = []
stockSet = AIModelHandler.selectAllStock()
for isin in stockSet:
    result.append(getRealtimePrice(isin))
pd.DataFrame(result)

NameError: name 'stockSet' is not defined